In [1]:
from javascript import require, On, Once, AsyncTask, once, off, globalThis
import json
import logging
import time
import vision

import building

MINECRAFT_VERSION = '1.20.2'

craft = require('./control_primitives/craftItem.js')
globalThis.craft = craft
explore = require('./control_primitives/exploreUntil.js')
globalThis.explore = explore
move = require('./control_primitives/goToPosition.js')
globalThis.move = move
kill = require('./control_primitives/killMob.js')
globalThis.kill = kill
collectPosition = require('./control_primitives/mineBlockPosition.js')
globalThis.collectPosition = collectPosition
collectType = require('./control_primitives/mineBlockType.js')
globalThis.collectType = collectType
pickupDroppedItem = require('./control_primitives/pickupDroppedItem.js')
globalThis.pickupDroppedItem = pickupDroppedItem
place = require('./control_primitives/placeItem.js')
globalThis.place = place
shoot = require('./control_primitives/shoot.js')
globalThis.shoot = shoot
smelt = require('./control_primitives/smeltItem.js')
globalThis.smelt = smelt
chest = require('./control_primitives/useChest.js')
globalThis.chest = chest

Vec3 = require('vec3').Vec3
globalThis.Vec3 = Vec3
mineflayer = require('mineflayer')
globalThis.mineflayer = mineflayer
pathfinder = require('mineflayer-pathfinder')
globalThis.pathfinder = pathfinder
pvp = require('mineflayer-pvp').plugin
globalThis.pvp = pvp
mcData = require('minecraft-data')(MINECRAFT_VERSION)
globalThis.mcData = mcData
armorManager = require("mineflayer-armor-manager")
globalThis.armorManager = armorManager
autoeat = require('mineflayer-auto-eat').plugin
globalThis.autoeat = autoeat
collectblock = require('mineflayer-collectblock').plugin
globalThis.collectblock = collectblock
hawkeye = require('minecrafthawkeye')
globalThis.hawkeye = hawkeye
toolPlugin = require('mineflayer-tool').plugin
globalThis.toolPlugin = toolPlugin
# mineflayerViewer = require('prismarine-viewer')

RANGE_GOAL = 1

# Fixing some crafting recipes involving planks
all_planks = [name for name in mcData.blocksByName if 'planks' in name]
crafting_table_list = []
bookshelves_list = []

stick_list = [] # [{ 'inShape': [int(mcData.itemsByName['bamboo']['id']), int(mcData.itemsByName['bamboo']['id'])], 'result': { 'id': 807, 'count': 1 } }]

shield_list = []
wood_sword_list = []
wood_pickaxe_list = []
wood_axe_list = []
wood_shovel_list = []
wood_hoe_list = []

for name in all_planks:
    crafting_table_list.append({ 'inShape': [[int(mcData.itemsByName[name]['id']), int(mcData.itemsByName[name]['id'])], [int(mcData.itemsByName[name]['id']), int(mcData.itemsByName[name]['id'])]], 'result': { 'id': 278, 'count': 1 } })
    bookshelves_list.append({ 'inShape': [[int(mcData.itemsByName[name]['id']), int(mcData.itemsByName[name]['id']), int(mcData.itemsByName[name]['id'])], [int(mcData.itemsByName['book']['id']), int(mcData.itemsByName['book']['id']), int(mcData.itemsByName['book']['id'])], [int(mcData.itemsByName[name]['id']), int(mcData.itemsByName[name]['id']), int(mcData.itemsByName[name]['id'])]], 'result': { 'id': 264, 'count': 1 } })
    
    stick_list.append({ 'inShape': [[int(mcData.itemsByName[name]['id'])], [int(mcData.itemsByName[name]['id'])]], 'result': { 'id': 807, 'count': 4 } })
    
    shield_list.append({ 'inShape': [[int(mcData.itemsByName[name]['id']), int(mcData.itemsByName['iron_ingot']['id']), int(mcData.itemsByName[name]['id'])], [int(mcData.itemsByName[name]['id']), int(mcData.itemsByName[name]['id']), int(mcData.itemsByName[name]['id'])], [None, int(mcData.itemsByName[name]['id']), None]], 'result': { 'id': 1116, 'count': 1 } })
    wood_sword_list.append({ 'inShape': [[int(mcData.itemsByName[name]['id'])], [int(mcData.itemsByName[name]['id'])], [int(mcData.itemsByName['stick']['id'])]], 'result': { 'id': 777, 'count': 1 } })
    wood_pickaxe_list.append({ 'inShape': [[int(mcData.itemsByName[name]['id']), int(mcData.itemsByName[name]['id']), int(mcData.itemsByName[name]['id'])], [None, int(mcData.itemsByName['stick']['id']), None], [None, int(mcData.itemsByName['stick']['id']), None]], 'result': { 'id': 779, 'count': 1 } })
    wood_axe_list.append({ 'inShape': [[int(mcData.itemsByName[name]['id']), int(mcData.itemsByName[name]['id'])], [int(mcData.itemsByName[name]['id']), int(mcData.itemsByName['stick']['id'])], [None, int(mcData.itemsByName['stick']['id'])]], 'result': { 'id': 780, 'count': 1 } })
    wood_shovel_list.append({ 'inShape': [[int(mcData.itemsByName[name]['id'])], [int(mcData.itemsByName['stick']['id'])], [int(mcData.itemsByName['stick']['id'])]], 'result': { 'id': 778, 'count': 1 } })
    wood_hoe_list.append({ 'inShape': [[int(mcData.itemsByName[name]['id']), int(mcData.itemsByName[name]['id'])], [None, int(mcData.itemsByName['stick']['id'])], [None, int(mcData.itemsByName['stick']['id'])]], 'result': { 'id': 781, 'count': 1 } })

mcData.recipes[mcData.itemsByName['crafting_table']['id']] = crafting_table_list
mcData.recipes[mcData.itemsByName['bookshelf']['id']] = bookshelves_list
mcData.recipes[mcData.itemsByName['stick']['id']] = stick_list
mcData.recipes[mcData.itemsByName['shield']['id']] = shield_list
mcData.recipes[mcData.itemsByName['wooden_sword']['id']] = wood_sword_list
mcData.recipes[mcData.itemsByName['wooden_pickaxe']['id']] = wood_pickaxe_list
mcData.recipes[mcData.itemsByName['wooden_axe']['id']] = wood_axe_list
mcData.recipes[mcData.itemsByName['wooden_shovel']['id']] = wood_shovel_list
mcData.recipes[mcData.itemsByName['wooden_hoe']['id']] = wood_hoe_list

# Load bot
def load_bot(username=None):
  # Create bot
  if not username:
    random_number = 176
    username = f'poo_bucket{random_number}'

  bot = mineflayer.createBot({ 
      'host': 'localhost',
      'port': 22222,
      'username': username, 
      'hideErrors': False 
  })

  once(bot, 'login')
  bot.chat('I spawned')
  #   mineflayerViewer(bot, { 'port': 3000, 'firstPerson': True })

  # Load plugins
  bot.loadPlugin(collectblock)
  bot.loadPlugin(hawkeye)
  bot.loadPlugin(pathfinder.pathfinder)
  bot.loadPlugin(pvp)
  bot.loadPlugin(toolPlugin)

  globalThis.mcData = mcData
  globalThis.pathfinder = pathfinder
  movements = pathfinder.Movements(bot, mcData)
  bot.pathfinder.setMovements(movements)

  # Auto armor
  bot.loadPlugin(armorManager)
  bot.armorManager.equipAll()

  # Auto eat
  bot.loadPlugin(autoeat)
  bot.autoEat.options.priority = 'foodPoints'
  bot.autoEat.options.startAt = 14
  bot.autoEat.options.bannedFood.push('golden_apple')
  
  return bot, movements

In [2]:
bot, movements = load_bot('poo_bucket176')
globalThis.bot = bot

[JSE] Mineflayer detected that you are using a deprecated event (physicTick)! Please use this event (physicsTick) instead.



In [3]:
import memory
import perception
import action

percept = perception.Module()
mem = memory.Module()
act = action.Module()

In [5]:
act.perform_action(bot, 'Craft a stone pickaxe.')

RateLimitError: Error code: 429 - {'error': {'message': 'You exceeded your current quota, please check your plan and billing details. For more information on this error, read the docs: https://platform.openai.com/docs/guides/error-codes/api-errors.', 'type': 'insufficient_quota', 'param': None, 'code': 'insufficient_quota'}}